In [2]:
#转换思路，用LSTM学习人类曲线后去预判，然后把LOSS当特征
import numpy as np
from sklearn.preprocessing import normalize
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
torch.manual_seed(1)

In [85]:
#global variables

batch_size = 12
sample_point = 129
niter = 25
TEST = False

In [6]:
def sort_track(track):
    #此函数用于将矩阵沿-1属性排序
    return track[np.lexsort(track.T)][0]
def get_track(s):
    i,moves,des,flag = s.split(' ')
    moves=moves.split(';')[:-1]
    moves=[move.split(',') for move in moves]
    moves=[[int(ll) for ll in l] for l in moves]
    des=[float(p) for p in des.split(',')]
    flag=int(flag)
#     print(i,moves[0],'->',des,flag)
    m = np.matrix(moves)
    m = m[np.lexsort(m.T)][0]
#     print(m)
    return (i,(m[0,0],m[0,1]), m, des, flag)
def get_test_track(s):
    i,moves,des = s.split(' ')
    moves=moves.split(';')[:-1]
    moves=[move.split(',') for move in moves]
    moves=[[int(ll) for ll in l] for l in moves]
    des=[float(p) for p in des.split(',')]
#     flag=int(flag)
#     print(i,moves[0],'->',des,flag)
    m = np.matrix(moves)
    m = m[np.lexsort(m.T)][0]
#     print(m)
    return (i,(m[0,0],m[0,1]), m, des)
    

In [7]:
def expand_track(track, max_sample=200):
    '''
    此函数用来扩充取样点，具体扩充方法是每两个点取似然中间点，符合x和y轴上速度相似的设定
    为了扩充至少需要三个点，来计算加速度。
    我认为，如果只有两个点，那无论什么模型都很难判断轨迹是否是robot，所以不如当作脏数据清洗掉
    输入为track的matrix,返回扩充后的matrix，shape'=(shape[0]*2-1, shape[1])
    '''
    max_exp = max_sample - track.shape[0]#最大扩充点数，使得扩充后的样点数量不大于max_sample
#     v = get_speed(track)
#     a = get_acceleration(track)
#     t = track[1:,-1] - track[:-1,-1]
    #加速度相等的推测点算法，结果轨迹不正常
#     Xb = track[1:-1,:-1] - np.multiply(v,t[:-1]/2) - np.multiply(a, np.square(t[:-1]/2))*0.5
#     Xb_e = track[-2,:-1] + np.multiply(v[-1],t[-1]/2) + np.multiply(a[-1], np.square(t[-1]/2))*0.5
#     to_exp = np.vstack([Xb, Xb_e])
    #速度相等的推测点算法，结果能接受
#     Xb = track[1:-1,:-1] + np.multiply(v,t[1:]/2)
#     Xb_s = track[1,:-1] - np.multiply(v[0],t[0]/2)
#     to_exp = np.vstack([Xb_s, Xb])
    #直接取中间点，对轨迹影响最小
    Xb = (track[:-1] + track[1:])/2
    to_exp = Xb
    expanded =  np.vstack([track, to_exp]) if to_exp.shape[0] <= max_exp else np.vstack([track, to_exp[:max_exp]])
    return sort_track(expanded)


In [8]:
def drop_track(track, max_sample=100):
    '''
    此函数用于将轨迹中超过数量的点抛弃
    抛弃规则是时间距离越近越容易被抛弃
    '''
    t = track[1:,-1] - track[:-1, -1]
    to_drop = track.shape[0] - max_sample
    assert to_drop >= 0
    for i in range(to_drop):
        track = track[np.ix_((np.argsort(track[:,-1],axis=0)>0).flat,[0,1,2])]
    return track


In [9]:
people = []
machine = []
test = []
with open('dsjtzs_txfz_training.txt') as f:
    people = list(filter(lambda l:l.strip().endswith('1'),f.readlines()))
    f.seek(0)
    machine = list(filter(lambda l:l.strip().endswith('0'),f.readlines()))
with open('dsjtzs_txfz_test1.txt') as f:
    test = f.readlines()

In [123]:
from random import shuffle
shuffle(people)
dataS1 = people
dataS2 = machine
sample = sample_point
X_train = np.empty((len(dataS1)+len(dataS2), sample, 3))
# dataB = np.empty(len(dataS2))
for i,d in enumerate(dataS1):
    index, start, track, des, flag = get_track(d)
    if track.shape[0] <2:
        X_train[i] = X_train[i-1]#除非第一个样本就只有一个点
        print("%s dropped"%index)
        continue
    while track.shape[0] < sample:
        track = expand_track(track, sample*2)
    X_train[i] = drop_track(track, sample)
for i,d in enumerate(dataS2):
    index, start, track, des, flag = get_track(d)
    if track.shape[0] <2:
        X_train[len(dataS1)+i] = X_train[len(dataS1)+i-1]
        print("%s dropped"%index)
        continue
    while track.shape[0] < sample:
        track = expand_track(track, sample*2)
    X_train[len(dataS1)+i] = drop_track(track, sample)
Y_train = np.vstack([np.zeros((len(dataS1),1)), 
                     np.ones((100,1))*1, np.ones((100,1))*2, 
                     np.ones((100,1))*3, np.ones((100,1))*4])

if TEST:
    X_test = np.empty((len(test), sample, 3))
    X_index = np.empty(len(test))
    for i,d in enumerate(test):
        index, start, track, des = get_test_track(d)
        if track.shape[0] <2:
            X_test[i] = X_test[i-1]
            print("%s dropped"%index)
            continue
        while track.shape[0] < sample:
            track = expand_track(track, sample*2)
        X_test[i] = drop_track(track, sample)
        X_index[i] = int(index)
    print(X_test.shape)
    
print(X_train.shape)
print(Y_train.shape)

493 dropped
2019 dropped
(3000, 129, 3)
(3000, 1)


In [124]:
from torch.utils.data import TensorDataset, DataLoader

_X_train = X_train[:,1:,:] - X_train[:,:-1,:]
_X_train = _X_train.transpose((0,2,1))
print(_X_train.shape)
dataset = TensorDataset(torch.Tensor(_X_train), torch.Tensor(Y_train))
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=4)

if TEST:
    _X_test = X_test[:,1:,:] - X_test[:,:-1,:]
    _X_test = _X_test.transpose((0,2,1))
    print(_X_test.shape)
    testset = TensorDataset(torch.Tensor(_X_test), torch.Tensor(X_index))
    testloader = DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=4)

(3000, 3, 128)


In [14]:
_X_train[:,:,2].max()

1221.0

In [131]:
import torch
from torch import nn
ndf = 64
# custom weights initialization called on netG and netD
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)

class _netD(nn.Module):
    def __init__(self):
        super(_netD,self).__init__()
        self.ConvLayer = nn.Sequential(
            # input is (nc) x 128
            nn.Conv1d(3, ndf, 4, 1, 0, bias=False),
            nn.InstanceNorm1d(ndf),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf) x128
            nn.Conv1d(ndf, ndf * 2, 4, 1, 0, bias=False),
            nn.InstanceNorm1d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*2) x 128
#             nn.Conv1d(ndf * 2, ndf * 4, 4, 1, 0, bias=False),
#             nn.InstanceNorm1d(ndf * 4),
#             nn.LeakyReLU(0.2, inplace=True),
#             # state size. (ndf*4) x 128
#             nn.Conv1d(ndf * 4, ndf * 8, 4, 1, 0, bias=False),
#             nn.InstanceNorm1d(ndf * 8),
#             nn.LeakyReLU(0.2, inplace=True),
#             # state size. (ndf*8) x 128
#             nn.Conv1d(ndf * 8, ndf * 16, 4, 1, 0, bias=False),
#             nn.InstanceNorm1d(ndf * 16),
#             nn.LeakyReLU(0.2, inplace=True),
#             nn.Conv1d(ndf * 16, 1, 4, 1, 0, bias=False),
        )
        self.FcLayer = nn.Sequential(
            nn.Linear(128*122,60*64),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(60*64,30*32),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(30*32,8*5),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(8*5,5),
#             nn.Softmax(),
#             nn.Linear(5,1),
        )
        self.ac = nn.Softmax()

    def forward(self, input):
        output = self.ConvLayer(input)
#         print(output.size())
        output = self.FcLayer(output.view(-1, 128*122))
#         print(output.size())
#         output = self.fc(output.view(-1,1))
#         output = self.ac(output.view(-1,1))
        return output


netD = _netD()
netD.apply(weights_init)
print(netD)

crossEntWeight = torch.Tensor([(len(dataS2)/len(dataS1)/4), 1, 1, 1, 1])
criterion = nn.CrossEntropyLoss(crossEntWeight)

_netD (
  (ConvLayer): Sequential (
    (0): Conv1d(3, 64, kernel_size=(4,), stride=(1,), bias=False)
    (1): InstanceNorm1d(64, eps=1e-05, momentum=0.1, affine=False)
    (2): LeakyReLU (0.2, inplace)
    (3): Conv1d(64, 128, kernel_size=(4,), stride=(1,), bias=False)
    (4): InstanceNorm1d(128, eps=1e-05, momentum=0.1, affine=False)
    (5): LeakyReLU (0.2, inplace)
  )
  (FcLayer): Sequential (
    (0): Linear (15616 -> 3840)
    (1): LeakyReLU (0.2, inplace)
    (2): Linear (3840 -> 960)
    (3): LeakyReLU (0.2, inplace)
    (4): Linear (960 -> 40)
    (5): LeakyReLU (0.2, inplace)
    (6): Linear (40 -> 5)
  )
  (ac): Softmax ()
)


In [132]:
inputs = torch.FloatTensor(batch_size, 3, 128)
label = torch.FloatTensor(batch_size)
inputs = autograd.Variable(inputs)
label = autograd.Variable(label.long())
optimizerD = optim.Adam(netD.parameters(), lr=0.0002, betas=(0.5, 0.999))

for epoch in range(niter):
    for i, data in enumerate(dataloader, 0):
        ############################
        # (1) Update D network: maximize log(D(x)) + log(1 - D(G(z)))
        ###########################
        # train with real
        netD.zero_grad()
        real_cpu, _ = data
        batch_size = real_cpu.size(0)
        inputs.data.resize_(real_cpu.size()).copy_(real_cpu)
        label.data.resize_(batch_size).copy_(_)

        output = netD(inputs)
#         print(output, label)
        errD = criterion(output, label.view(-1))
        errD.backward()
        D_x = output.data.mean()

        optimizerD.step()
        print('[%d/%d][%d/%d] Loss_D: %.4f D(x): %.4f'
              % (epoch, niter, i, len(dataloader),
                 errD.data[0], D_x))

[0/25][0/250] Loss_D: 1.6002 D(x): 0.0011
[0/25][1/250] Loss_D: 1.6609 D(x): -0.0223
[0/25][2/250] Loss_D: 0.9242 D(x): -0.0341
[0/25][3/250] Loss_D: 2.0833 D(x): -0.1213
[0/25][4/250] Loss_D: 0.2140 D(x): -0.0902
[0/25][5/250] Loss_D: 0.0618 D(x): -0.1303
[0/25][6/250] Loss_D: 0.0248 D(x): -0.2135
[0/25][7/250] Loss_D: 4.4676 D(x): -0.2566
[0/25][8/250] Loss_D: 1.2910 D(x): -0.0814
[0/25][9/250] Loss_D: 1.2234 D(x): -0.0498
[0/25][10/250] Loss_D: 1.0139 D(x): -0.0455
[0/25][11/250] Loss_D: 1.5964 D(x): -0.0884
[0/25][12/250] Loss_D: 1.8100 D(x): -0.1281
[0/25][13/250] Loss_D: 1.2052 D(x): -0.1013
[0/25][14/250] Loss_D: 1.8286 D(x): -0.1661
[0/25][15/250] Loss_D: 1.2846 D(x): -0.1231
[0/25][16/250] Loss_D: 1.7909 D(x): -0.1002
[0/25][17/250] Loss_D: 1.1944 D(x): -0.0643
[0/25][18/250] Loss_D: 1.3012 D(x): -0.0887


Process Process-409:
Process Process-412:
Process Process-410:
Traceback (most recent call last):
Process Process-411:
  File "/home/miller/anaconda3/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/home/miller/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
  File "/home/miller/anaconda3/lib/python3.6/multiprocessing/queues.py", line 342, in get
    with self._rlock:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/miller/anaconda3/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 35, in _worker_loop
    r = index_queue.get()
  File "/home/miller/anaconda3/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/home/miller/anaconda3/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/home/miller/anaconda3/lib/python3.6/m

KeyboardInterrupt: 

In [122]:
# netD.load_state_dict(torch.load('%s/netD_epoch_%d.pth' % ('model', epoch)))

# predict test
# dataiter = iter(testloader)
# with open('result/torch-conv.txt', 'w') as f:
#     for i,data in enumerate(testloader,0):
#         tracks, index = data
#         tracks = autograd.Variable(tracks)
#         for c,t in enumerate(netD(tracks)>0.5):
#             if t.data[0]:
#                 f.write("%d\n"%index[c])
# print(index)

# preidct train
dataiter = iter(dataloader)
tracks, label = dataiter.next()
tracks = autograd.Variable(tracks)
outputs = netD(tracks)
# print(outputs)
_, predicted = torch.max(outputs.data, 1)
print(predicted)
# print(netD(tracks))
print(label)


# save model
# torch.save(netD.state_dict(), '%s/netD_5_epoch_%d.pth' % ('model', epoch))


 4
 2
 3
 4
 3
 1
 1
 0
 0
 2
 1
 4
[torch.LongTensor of size 12]


 4
 2
 3
 4
 3
 1
 1
 0
 0
 2
 1
 4
[torch.FloatTensor of size 12]



In [38]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalAveragePooling1D, MaxPooling1D
from keras.layers.normalization import BatchNormalization

model = Sequential()
model.add(Conv1D(2, 2, activation='relu', input_shape=(100, 3)))
model.add(BatchNormalization(axis=1))
model.add(Conv1D(64, 3, activation='relu'))
model.add(BatchNormalization(axis=1))
model.add(MaxPooling1D(3))
model.add(Conv1D(128, 3, activation='relu'))
model.add(BatchNormalization(axis=1))
model.add(Conv1D(128, 3, activation='relu'))
model.add(BatchNormalization(axis=1))
model.add(GlobalAveragePooling1D())
# model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])
_X_train = np.vstack([X_train[100:200], X_train[-100:]])
_X_train = _X_train[:,1:,:] - _X_train[:,:-1,:]
_Y_train = np.vstack([Y_train[100:200], Y_train[-100:]])
model.fit(_X_train, _Y_train, batch_size=16, epochs=25,shuffle=True,validation_split=0.1)
# score = model.evaluate(x_test, y_test, batch_size=16)

Train on 180 samples, validate on 20 samples
Epoch 1/25
180/180 [==============================] - 7s - loss: 0.5990 - acc: 0.7500 - val_loss: 0.6557 - val_acc: 0.8000

In [ ]:
_test[_test>10000] =0 
result = np.hstack((svc.predict(_test).reshape(-1,1),_test_index))

In [ ]:
# score = model.evaluate(X_train, Y_train, batch_size=16)
_X_test = X_train[:,1:] - X_train[:,:-1] 
pred = model.predict(_X_test)
# print(pred.shape)
# np.save("model/m5.npy",pred)
# np.count_nonzero(pred>0.1)
# model.save_weights('m1.hdfs')

# with open('m1.struct', 'w') as f:
#     f.write(model.to_json())
    
# with open('m5.struct', 'r') as f:
#     model = model_from_json(f.read())
# model.load_weights('m5.hdfs')
print(pred[-100:])

In [9]:
X_train = dataA
X_trainB = dataB

Y_trainB = np.zeros(dataB.shape[0]).reshape((-1,1))
Y_train = np.ones(dataA.shape[0]).reshape((-1,1))
print(dataA[0].shape)

(56, 3)


In [ ]:
model.fit(np.expand_dims(X_train[0], axis=2), Y_train[0], batch_size=1, epochs=1)

In [106]:
class LSTMTagger(nn.Module):
    
    def __init__(self, input_dim, hidden_dim, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim
        
        self.lstm = nn.LSTM(input_dim, hidden_dim)
#         self.dropout = nn.Dropout(0.5)
        # The linear layer that maps from hidden state space to tag space
        self.hidden2pos = nn.Linear(hidden_dim, tagset_size)
        self.active = nn.Sigmoid()
        self.hidden = self.init_hidden()
        
    def init_hidden(self):
        # Before we've done anything, we dont have any hidden state.
        # Refer to the Pytorch documentation to see exactly why they have this dimensionality.
        # The axes semantics are (num_layers, minibatch_size, hidden_dim)
        return (autograd.Variable(torch.zeros(1, 1, self.hidden_dim)),
                autograd.Variable(torch.zeros(1, 1, self.hidden_dim)))
        
    def forward(self, track):
#         print(track)
        lstm_out, self.hidden = self.lstm(track.view(len(track), 1, -1), self.hidden)
#         dropped = self.dropout(lstm_out.view(len(track), -1))
        pos_space = self.hidden2pos(lstm_out.view(len(track), -1))
        return self.active(pos_space)[-1]

In [94]:
input_dim,hidden_dim, tagset_size = 3,8,1
def init_hidden():
    # Before we've done anything, we dont have any hidden state.
    # Refer to the Pytorch documentation to see exactly why they have this dimensionality.
    # The axes semantics are (num_layers, minibatch_size, hidden_dim)
    return (autograd.Variable(torch.zeros(1, 1, hidden_dim)),
            autograd.Variable(torch.zeros(1, 1, hidden_dim)))

hidden = init_hidden()
ll = nn.Linear(hidden_dim, tagset_size)
lo = nn.Conv1d(hidden_dim, tagset_size,2)
active = nn.Sigmoid()

track = dataA[0]
track = normalize(track,axis=0)
m_lstm = nn.LSTM(3, 8)
inputs = autograd.Variable(torch.Tensor(track)).contiguous()
print(inputs)
lstm_out, hidden = m_lstm(inputs.view(len(inputs), 1, -1), hidden)
pos = ll(lstm_out.view(len(lstm_out), -1))
# print(active(pos)[-1])
targets = autograd.Variable(torch.Tensor([1])).contiguous()
# print(targets)
# inputs

Variable containing:
 0.0342  0.1319  0.0161
 0.0355  0.1319  0.0173
 0.0375  0.1326  0.0193
 0.0403  0.1326  0.0204
 0.0484  0.1326  0.0227
 0.0565  0.1326  0.0252
 0.0653  0.1326  0.0273
 0.0701  0.1326  0.0297
 0.0755  0.1326  0.0320
 0.0795  0.1326  0.0342
 0.0822  0.1332  0.0366
 0.0856  0.1332  0.0389
 0.0904  0.1332  0.0413
 0.0951  0.1332  0.0436
 0.1026  0.1332  0.0464
 0.1107  0.1332  0.0490
 0.1195  0.1332  0.0504
 0.1242  0.1332  0.0528
 0.1269  0.1332  0.0558
 0.1283  0.1332  0.0573
 0.1290  0.1332  0.0625
 0.1310  0.1332  0.0649
 0.1330  0.1339  0.0669
 0.1405  0.1345  0.0688
 0.1425  0.1352  0.0712
 0.1432  0.1352  0.0901
 0.1439  0.1352  0.0946
 0.1446  0.1352  0.1030
 0.1452  0.1352  0.1036
 0.1466  0.1352  0.1058
 0.1466  0.1345  0.1083
 0.1473  0.1345  0.1178
 0.1479  0.1345  0.1256
 0.1493  0.1345  0.1279
 0.1500  0.1345  0.1289
 0.1513  0.1345  0.1311
 0.1527  0.1345  0.1334
 0.1540  0.1345  0.1358
 0.1547  0.1345  0.1380
 0.1554  0.1345  0.1404
 0.1561  0.1345  0.

/home/miller/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by the normalize function.
  warnings.warn(msg, _DataConversionWarning)


In [107]:
model = LSTMTagger(3, 10, 1)
loss_function = nn.BCELoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

In [91]:
tensor = torch.LongTensor(X_train[0])
inputs = torch.LongTensor(X_train[0][:,2])
targets = torch.LongTensor(X_train[0][:,0:2])
# print(tensor.size(), targets.size())
# print(X_train[0])
# (targets.contiguous()).view(-1,2)

In [108]:
def train(model, tracks, target):
    # Step 1. Remember that Pytorch accumulates gradients.  We need to clear them out
    # before each instance
    model.zero_grad()
    tracks = normalize(tracks, axis=0)

    # Also, we need to clear out the hidden state of the LSTM, detaching it from its
    # history on the last instance.
    model.hidden = model.init_hidden()

    # Step 2. Get our inputs ready for the network, that is, turn them into Variables
    # of word indices.
    inputs = tracks
    targets = [target]
    inputs = autograd.Variable(torch.Tensor(inputs)).contiguous()
    targets = autograd.Variable(torch.Tensor(targets)).contiguous()

    # Step 3. Run our forward pass.
    tag_scores = model(inputs.float())
    loss = loss_function(tag_scores, targets.float())
    loss.backward(retain_variables=True)
    optimizer.step()
    return loss

for epoch in range(2): # again, normally you would NOT do 300 epochs, it is toy data
    for i,tracks in enumerate(X_train):
        if i < 400:
            train(model, tracks, 1)
            loss = train(model, dataB[i], 0)
        if i%50 == 0:
            print(loss)

/home/miller/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by the normalize function.
  warnings.warn(msg, _DataConversionWarning)


Variable containing:
 0.6276
[torch.FloatTensor of size 1]

Variable containing:
 0.7054
[torch.FloatTensor of size 1]

Variable containing:
 0.7021
[torch.FloatTensor of size 1]

Variable containing:
 0.6999
[torch.FloatTensor of size 1]

Variable containing:
 0.7094
[torch.FloatTensor of size 1]

Variable containing:
 0.7124
[torch.FloatTensor of size 1]

Variable containing:
 0.6995
[torch.FloatTensor of size 1]

Variable containing:
 0.6935
[torch.FloatTensor of size 1]

Variable containing:
 0.7071
[torch.FloatTensor of size 1]

Variable containing:
 0.7071
[torch.FloatTensor of size 1]

Variable containing:
 0.7071
[torch.FloatTensor of size 1]

Variable containing:
 0.7071
[torch.FloatTensor of size 1]

Variable containing:
 0.7071
[torch.FloatTensor of size 1]

Variable containing:
 0.7071
[torch.FloatTensor of size 1]

Variable containing:
 0.7071
[torch.FloatTensor of size 1]

Variable containing:
 0.7071
[torch.FloatTensor of size 1]

Variable containing:
 0.7071
[torch.Floa

In [140]:
inputs.size()

torch.Size([193, 4])

In [115]:
import pandas as pd
import numpy as np
import random
from keras.models import Sequential, model_from_json
from keras.layers import Dense, LSTM, Dropout

class NeuralNetwork():
    def __init__(self, **kwargs):
        """
        :param **kwargs: output_dim=4: output dimension of LSTM layer; 
        activation_lstm='tanh': activation function for LSTM layers; 
        activation_dense='relu': activation function for Dense layer; 
        activation_last='sigmoid': activation function for last layer; 
        drop_out=0.2: fraction of input units to drop; 
        np_epoch=10, the number of epoches to train the model. 
            epoch is one forward pass and one backward pass of all the training examples; 
        batch_size=32: number of samples per gradient update. 
            The higher the batch size, the more memory space you'll need; 
        loss='mean_square_error': loss function; 
        optimizer='rmsprop'
        """
        self.output_dim = kwargs.get('output_dim', 8)
        self.activation_lstm = kwargs.get('activation_lstm', 'relu')
        self.activation_dense = kwargs.get('activation_dense', 'relu')
        self.activation_last = kwargs.get('activation_last', 'softmax')    # softmax for multiple output
        self.dense_layer = kwargs.get('dense_layer', 2)     # at least 2 layers
        self.lstm_layer = kwargs.get('lstm_layer', 2)
        self.drop_out = kwargs.get('drop_out', 0.2)
        self.nb_epoch = kwargs.get('nb_epoch', 10)
        self.batch_size = kwargs.get('batch_size', 100)
        self.loss = kwargs.get('loss', 'categorical_crossentropy')
        self.optimizer = kwargs.get('optimizer', 'rmsprop')

    def NN_model(self, trainX, trainY, testX, testY):
        """
        :param trainX: training data set
        :param trainY: expect value of training data
        :param testX: test data set
        :param testY: epect value of test data
        :return: model after training
        """
        print("Training model is LSTM network!")
        input_dim = trainX[1].shape[1]
        output_dim = trainY.shape[1] # one-hot label
        # print predefined parameters of current model:
        model = Sequential()
        # applying a LSTM layer with x dim output and y dim input. Use dropout parameter to avoid overfitting
        model.add(LSTM(output_dim=self.output_dim,
                       input_dim=input_dim,
                       activation=self.activation_lstm,
                       dropout_U=self.drop_out,
                       return_sequences=True))
        for i in range(self.lstm_layer-2):
            model.add(LSTM(output_dim=self.output_dim,
                       input_dim=self.output_dim,
                       activation=self.activation_lstm,
                       dropout_U=self.drop_out,
                       return_sequences=True))
        # argument return_sequences should be false in last lstm layer to avoid input dimension incompatibility with dense layer
            model.add(LSTM(output_dim=self.output_dim,
                       input_dim=self.output_dim,
                       activation=self.activation_lstm,
                       dropout_U=self.drop_out))
        for i in range(self.dense_layer-1):
            model.add(Dense(output_dim=self.output_dim,
                        activation=self.activation_last))
            model.add(Dense(output_dim=output_dim,
                        input_dim=self.output_dim,
                        activation=self.activation_last))
        # configure the learning process
            model.compile(loss=self.loss, optimizer=self.optimizer, metrics=['accuracy'])
        # train the model with fixed number of epoches
            model.fit(x=trainX, y=trainY, nb_epoch=self.nb_epoch, batch_size=self.batch_size, validation_data=(testX, testY))
        # store model to json file
            model_json = model.to_json()
        with open(model_path, "w") as json_file:
            json_file.write(model_json)
        # store model weights to hdf5 file
        if model_weight_path:
            if os.path.exists(model_weight_path):
                os.remove(model_weight_path)
            model.save_weights(model_weight_path) # eg: model_weight.h5
        return model

Using TensorFlow backend.


In [111]:

model = NeuralNetwork(output_dim=3, batch_size=1)

(400,)
